## String Matching Simple Chatbot

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("/content/Training Data.csv")

In [5]:
df.head()

,instruction (string),output (string)
0,What is a normal menstrual cycle length?,A normal menstrual cycle typically ranges from...
1,What are common causes of irregular periods?,Common causes of irregular periods include hor...
2,How can I alleviate menstrual cramps?,Menstrual cramps can be alleviated through var...
3,What are the signs of a heavy menstrual flow?,Signs of a heavy menstrual flow include soakin...
4,Is it normal to experience mood swings during ...,"Yes, it is common for some individuals to expe..."


In [6]:
df.shape

(530, 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   instruction (string)  530 non-null    object
 1   output (string)       530 non-null    object
dtypes: object(2)
memory usage: 8.4+ KB


In [8]:
df['instruction (string)'] = df['instruction (string)'].str.lower()

In [9]:
def string_matching(user_input):
    user_input = user_input.lower()
    for index, row in df.iterrows():
        if user_input == row['instruction (string)']:
            return row['output (string)']
    return "Sorry, I couldn't find a matching answer."

In [27]:
user_input = input("User: ")
response = string_matching(user_input)
print("Bot:", response)

User: How can I alleviate menstrual cramps?
Bot: Menstrual cramps can be alleviated through various methods including over-the-counter pain relievers, applying heat to the abdomen, gentle exercise, relaxation techniques, and dietary changes.


In [10]:
user_input = input("User: ")
response = string_matching(user_input)
print("Bot:", response)

User: what is menstrual cycle ?
Bot: Sorry, I couldn't find a matching answer.


## Limitations : exact question to be given
## Feature : Fast
# Fuzzy Matching

In [31]:
!pip install fuzzywuzzy
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.1 MB/s eta 0:00:00


In [11]:
from fuzzywuzzy import process
def fuzzy_matching(user_input):
    user_input = user_input.lower()
    best_match = process.extractOne(user_input, df['instruction (string)'])[0]
    similarity_score = process.extractOne(user_input, df['instruction (string)'])[1]
    if similarity_score > 90:  # Adjust threshold as needed
        return df[df['instruction (string)'] == best_match]['output (string)'].iloc[0]
    else:
        return "Sorry, I couldn't find a matching answer."


In [49]:
user_input = input("User: ")
response = fuzzy_matching(user_input)
print("Bot:", response)

User: What is a menstrual cycle length?
Bot: A normal menstrual cycle typically ranges from 21 to 35 days, with the average cycle lasting around 28 days.


## Limitations : similarity score can make wrong answers

# NLP Model

# 1. Using Spacy

In [52]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.0.0/en_core_web_lg-3.0.0.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 9.2 MB/s eta 0:00:00
  Created wheel for en-core-web-lg: filename=en_core_web_lg-3.0.0-py3-none-any.whl size=778750474 sha256=dc15dec51d8ab04d6c476f1301d436f1b4beec47c0a97acda52d4d1b60a25fba
  Stored in directory: /root/.cache/pip/wheels/4d/30/74/df255edbc5dbeb5c2e0ce666ac4989fec3b3289552889d531a
Successfully built en-core-web-lg
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.2
    Uninstalling wasabi-1.1.2:
      Successfully uninstalled wasabi-1.1.2
  Attempt

In [1]:
import spacy


nlp = spacy.load('en_core_web_lg')

def nlp_matching(user_input):

    user_tokens = [token.text.lower() for token in nlp(user_input) if not token.is_punct and not token.is_stop]


    max_similarity = 0
    best_answer = None
    for question in df['instruction (string)']:
        question_tokens = [token.text.lower() for token in nlp(question) if not token.is_punct and not token.is_stop]
        similarity = nlp(' '.join(user_tokens)).similarity(nlp(' '.join(question_tokens)))
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = df.loc[df['instruction (string)'] == question, 'output (string)'].values[0]

    if max_similarity >= 0.7:  # Adjust threshold as needed
        return best_answer
    else:
        return "Sorry, I couldn't find a matching answer."



/usr/local/lib/python3.10/dist-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


In [12]:
user_input = input("User: ")
response = nlp_matching(user_input)
print("Bot:", response)


User: what is menstrual cycle ?
Bot: The menstrual cycle consists of four main phases: the follicular phase, ovulation, the luteal phase, and menstruation. During the follicular phase, follicle-stimulating hormone (FSH) stimulates follicles in the ovary to mature. Ovulation occurs when a mature egg is released from the ovary. The luteal phase follows ovulation, during which the ruptured follicle forms the corpus luteum and secretes progesterone. If fertilization does not occur, the uterine lining sheds during menstruation, marking the end of the cycle.


## Training

In [15]:
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_lg')

def train_and_save_model(df):
    output_dir = "/content/Model"
    nlp.to_disk(output_dir)

def nlp_matching(user_input):

    user_tokens = [token.text.lower() for token in nlp(user_input) if not token.is_punct and not token.is_stop]

    max_similarity = 0
    best_answer = None
    for question in df['instruction (string)']:
        question_tokens = [token.text.lower() for token in nlp(question) if not token.is_punct and not token.is_stop]
        similarity = nlp(' '.join(user_tokens)).similarity(nlp(' '.join(question_tokens)))
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = df.loc[df['instruction (string)'] == question, 'output (string)'].values[0]

    if max_similarity >= 0.7:  # Adjust threshold as needed
        return best_answer
    else:
        return "Sorry, I couldn't find a matching answer."


df = pd.read_csv('/content/Training Data.csv')

train_and_save_model(df)



## using testing dataset

In [16]:
import spacy
import pandas as pd


output_dir = "/content/Model"
nlp = spacy.load(output_dir)

# Load your testing dataset
testing_df = pd.read_csv('/content/Testing Data.csv')

def nlp_matching(user_input, df):
    user_tokens = [token.text.lower() for token in nlp(user_input) if not token.is_punct and not token.is_stop]

    max_similarity = 0
    best_answer = None

    for question in df['instruction (string)']:
        question_tokens = [token.text.lower() for token in nlp(question) if not token.is_punct and not token.is_stop]


        similarity = nlp(' '.join(user_tokens)).similarity(nlp(' '.join(question_tokens)))

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = df.loc[df['instruction (string)'] == question, 'output (string)'].values[0]

    if max_similarity >= 0.7:  # Adjust threshold as needed
        return best_answer
    else:
        return "Sorry, I couldn't find a matching answer."



In [17]:
testing_df.head()

,instruction (string),output (string)
0,What is the physiological process behind menst...,Menstruation is the result of complex hormonal...
1,How do hormonal fluctuations throughout the me...,"Hormonal fluctuations, particularly in estroge..."
2,What are some underlying causes of menstrual i...,Menstrual irregularities can stem from various...
3,Can you explain the role of prostaglandins in ...,Prostaglandins are hormone-like substances rel...
4,What are some lifestyle modifications that can...,"Implementing regular exercise, maintaining a b..."


## Evaluating

In [18]:

correct_predictions = 0
total_examples = len(testing_df)

for index, row in testing_df.iterrows():
    user_input = row['instruction (string)']
    expected_answer = row['output (string)']


    predicted_answer = nlp_matching(user_input, testing_df)


    if predicted_answer == expected_answer:
        correct_predictions += 1


accuracy = correct_predictions / total_examples
print("Accuracy:", accuracy)

Accuracy: 1.0
